In [42]:
# import libraries
import pandas as pd
import glob

# select demand for victoria and sort in order of date
demand_vic = glob.glob("/Users/jiehni/monash/ads1002/ADS1002-project/Data/Demand/*_VIC*.csv")
demand_vic.sort()

# read demand data files and combine
df_list = [pd.read_csv(f) for f in demand_vic]
if df_list:
    demand_df = pd.concat(df_list, ignore_index=True)
    print(demand_df.head())
else:
    print("No CSV files were loaded.")

# rename and drop columns
demand_df = demand_df.rename(columns={
    "SETTLEMENTDATE": "Datetime",
    "TOTALDEMAND": "Total Demand"
})
demand_df = demand_df.drop(columns=["REGION", "PERIODTYPE"])

print(demand_df.head())

  REGION    SETTLEMENTDATE  TOTALDEMAND    RRP PERIODTYPE
0   VIC1  2000/01/01 00:30   4419.03667   9.97      TRADE
1   VIC1  2000/01/01 01:00   4312.54000  12.58      TRADE
2   VIC1  2000/01/01 01:30   4599.70167  13.22      TRADE
3   VIC1  2000/01/01 02:00   4478.99000  12.90      TRADE
4   VIC1  2000/01/01 02:30   4228.81000  12.40      TRADE
           Datetime  Total Demand    RRP
0  2000/01/01 00:30    4419.03667   9.97
1  2000/01/01 01:00    4312.54000  12.58
2  2000/01/01 01:30    4599.70167  13.22
3  2000/01/01 02:00    4478.99000  12.90
4  2000/01/01 02:30    4228.81000  12.40


In [43]:
# read weather data for melbourne victoria and combine
weather1_df = pd.read_csv("/Users/jiehni/monash/ads1002/ADS1002-project/Data/Temperature/HM01X_Data_086071_999999999743964.txt")
weather2_df = pd.read_csv("/Users/jiehni/monash/ads1002/ADS1002-project/Data/Temperature/HM01X_Data_086338_999999999743964.txt")
weather_df = pd.concat([weather1_df, weather2_df])

# combine datetime into one variable with the correct format
weather_df['Datetime'] = pd.to_datetime({
    'year': weather_df['Year Month Day Hour Minutes in YYYY'],
    'month': weather_df['MM'],
    'day': weather_df['DD'],
    'hour': weather_df['HH24'],
    'minute': weather_df['MI format in Local time']
})
weather_df['Datetime'] = weather_df['Datetime'].dt.strftime('%Y/%m/%d %H:%M')

# rename columns
weather_df = weather_df.rename(columns={
    "Precipitation since 9am local time in mm": "Precipitation (mm)",
    "Air Temperature in degrees C": "Air Temp (C)",
    "Relative humidity in percentage %": "Humidity (%)",
    "Wind speed in km/h": "Wind Speed (km/h)"
})

# convert variables to int
cols_to_convert = ["Precipitation (mm)", "Air Temp (C)", "Humidity (%)", "Wind Speed (km/h)"]

weather_df[cols_to_convert] = weather_df[cols_to_convert].apply(
    pd.to_numeric, errors='coerce'
)

print(weather_df.columns)
print(weather_df.head())

/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_65013/1965025615.py:2: DtypeWarning: Columns (12,14,18,20,22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  weather1_df = pd.read_csv("/Users/jiehni/monash/ads1002/ADS1002-project/Data/Temperature/HM01X_Data_086071_999999999743964.txt")
/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_65013/1965025615.py:3: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  weather2_df = pd.read_csv("/Users/jiehni/monash/ads1002/ADS1002-project/Data/Temperature/HM01X_Data_086338_999999999743964.txt")


Index(['hm', 'Station Number', 'Year Month Day Hour Minutes in YYYY', 'MM',
       'DD', 'HH24', 'MI format in Local time',
       'Year Month Day Hour Minutes in YYYY.1', 'MM.1', 'DD.1', 'HH24.1',
       'MI format in Local standard time', 'Precipitation (mm)',
       'Quality of precipitation since 9am local time', 'Air Temp (C)',
       'Quality of air temperature', 'Wet bulb temperature in degrees C',
       'Quality of Wet bulb temperature', 'Dew point temperature in degrees C',
       'Quality of dew point temperature', 'Humidity (%)',
       'Quality of relative humidity', 'Wind Speed (km/h)',
       'Wind speed quality', 'Wind direction in degrees true',
       'Wind direction quality',
       'Speed of maximum windgust in last 10 minutes in  km/h',
       'Quality of speed of maximum windgust in last 10 minutes',
       'Mean sea level pressure in hPa', 'Quality of mean sea level pressure',
       'Station level pressure in hPa', 'Quality of station level pressure',
       'AW

In [47]:
# merge demand and weather data
mergedvic_df = pd.merge(
    demand_df,
    weather_df[["Datetime", "Precipitation (mm)", "Air Temp (C)", "Humidity (%)", "Wind Speed (km/h)"]],
    on='Datetime',
    how='inner'
)
print(mergedvic_df.head())

# precip is only recorded after 9am
# fill not recorded values as NR for ease of use during analysis and modelling
mergedvic_df["Precipitation (mm)"] = mergedvic_df["Precipitation (mm)"].fillna("NR")

# drop invalid and empty values
mergedvic_df = mergedvic_df.drop_duplicates()
mergedvic_df = mergedvic_df.dropna()

print(mergedvic_df.head())

# save cleaned data to new file
mergedvic_df.to_csv("VIC_clean.csv", index=False)

           Datetime  Total Demand    RRP  Precipitation (mm)  Air Temp (C)  \
0  2000/01/01 01:30    4599.70167  13.22                 NaN          13.7   
1  2000/01/01 02:00    4478.99000  12.90                 NaN          13.6   
2  2000/01/01 02:30    4228.81000  12.40                 NaN          13.3   
3  2000/01/01 03:00    3989.10000   9.45                 NaN          14.1   
4  2000/01/01 03:30    3783.93167   4.82                 NaN          13.5   

   Humidity (%)  Wind Speed (km/h)  
0          65.0                0.0  
1          63.0                0.0  
2          68.0                0.0  
3          60.0                3.6  
4          67.0                1.8  
           Datetime  Total Demand    RRP Precipitation (mm)  Air Temp (C)  \
0  2000/01/01 01:30    4599.70167  13.22                 NR          13.7   
1  2000/01/01 02:00    4478.99000  12.90                 NR          13.6   
2  2000/01/01 02:30    4228.81000  12.40                 NR          13.3   
3